In [5]:
import logging
import pandas as pd
import os
import json
import time

In [6]:
from retrieve_data_by_query import RetrieveDataByQuery
t = RetrieveDataByQuery(sessionid="65955050144%3A6hS26w11VRkHbU%3A24%3AAYeTigtAMpGXCp8sOwNzqCe29YQYygcEAfegg7eTjg")
t.retrieve_follower_by_userid_after("1796326818",str(25*0))

{'errors': [{'message': 'execution error',
   'path': ['fetch__XDTUserDict'],
   'severity': 'CRITICAL'}],
 'data': None,
 'extensions': {'is_final': True},
 'status': 'ok'}

In [7]:
# query = "Artificial Intelligence"

In [8]:
def get_follower_data(data: list):
    try:
        return [
            {
                "pk": node.get("pk", ""),
                "username": node.get("username", ""),
                "follower_count": node.get("follower_count", "")
            }
            for d in data
            if (node := d.get("node", {}))
        ]
    except Exception as e:
        raise RuntimeError(f"Errore durante l'elaborazione dei dati dei follower: {e}")


progress_tracker = {}

def save_progress(userid: str, batch_number: int, results: list):
    """Salva il batch_number corrente e i risultati ottenuti per un determinato userid."""
    progress_tracker[userid] = {"batch_number": batch_number, "results": results}
    with open("progress2.json", "w") as f:
        json.dump(progress_tracker, f)

def load_progress(userid: str):
    """Carica il batch_number e i risultati salvati per un determinato userid."""
    try:
        with open("progress2.json", "r") as f:
            data = json.load(f)
            return data.get(userid, {"batch_number": 1, "results": []})
    except (FileNotFoundError, json.JSONDecodeError):
        return {"batch_number": 1, "results": []}
def remove_progress(userid: str):
    """Rimuove il progresso salvato per un determinato userid."""
    try:
        with open("progress2.json", "r") as f:
            data = json.load(f)
        
        if userid in data:
            del data[userid]
            with open("progress2.json", "w") as f:
                json.dump(data, f)
    except (FileNotFoundError, json.JSONDecodeError):
        pass
    
def get_all_follower(userid: str):
    progress = load_progress(userid)
    batch_number = progress["batch_number"]
    results = progress["results"]
    allpk = [res["pk"] for res in results]
    try:
        # if batch_number == 1:
        #     response = t.retrieve_follower_by_userid(userid)
        #     time.sleep(2)
        #     data = response.get("data", {}).get("user", {}).get("followers", {})
        #     if data.get("edges", []) == []:
        #         return [{
        #             "pk": userid,
        #             "username": None,
        #             "follower_count": None
        #         }]
        #     d_out = get_follower_data(data.get("edges", []))
        #     for res in d_out:
        #         allpk.append(res["pk"])
        #     results.extend(d_out)
        #     has_next_page = data.get("page_info", {}).get("has_next_page", False)
        # else:
        #     has_next_page = True
        has_next_page = True
        while has_next_page:
            try:
                print("Pagina:", batch_number)
                time.sleep(2)
                response = t.retrieve_follower_by_userid_after(userid, str(25 * batch_number-1))
                batch_number += 1
                data = response.get("data", {}).get("fetch__XDTUserDict", {}).get("followers", {})
                if data.get("edges", []) == []:
                    return [{
                        "pk": userid,
                        "username": None,
                        "follower_count": None
                    }]
                d_out = get_follower_data(data.get("edges", []))
                for res in d_out:
                    if res["pk"] in allpk:
                        remove_progress(userid)
                        return results
                    allpk.append(res["pk"])
                results.extend(d_out)
                
                has_next_page = data.get("page_info", {}).get("has_next_page", False)
                
                # Salva il progresso dopo ogni batch
                save_progress(userid, batch_number, results)
                
            except Exception as e:
                save_progress(userid, batch_number, results)  # Salva il batch e i risultati in caso di errore
                raise RuntimeError(f"Errore nel recupero della paginazione dei follower: {e}")
    
    except Exception as e:
        save_progress(userid, batch_number, results)  # Salva il batch e i risultati in caso di errore generale
        raise RuntimeError(f"Errore generale nel recupero dei follower per l'utente {userid}: {e}")
    remove_progress(userid)
    return results




def get_post_data(posts: list):
    results = []
    for post in posts:
        dati = post["node"]["thread"]["thread_items"]
        for p in dati:
            d = p['post']
            if int(d.get("like_count", 0)) < 100:
                continue
            userpk = d.get("user", {}).get("pk", "")
            caption = d.get("caption", {})
            caption_text = caption.get("text", "") if caption else ""
            if caption_text == "":
                continue
            results.append({
                "pk": d.get("pk", ""),
                "user_pk": userpk,
                "caption": caption_text,
                "like_count": int(d.get("like_count", 0)),
                "taken_at": d.get("taken_at", "")
            })
    return results

In [9]:


# results = []
# response = t.retrieve_thread_by_query(query)
# data = response.get("data", {}).get("searchResults", {})
# results.extend(get_post_data(data.get("edges", [])))
# has_next_page = data.get("page_info", {}).get("has_next_page", False)
# after = data.get("page_info", {}).get("end_cursor")
# i = 1000

# while has_next_page and i > 0:
#     response = t.retrieve_thread_by_query_after(query, after)
#     data = response.get("data", {}).get("searchResults", {})
#     results.extend(get_post_data(data.get("edges", [])))
#     has_next_page = data.get("page_info", {}).get("has_next_page", False)
#     after = data.get("page_info", {}).get("end_cursor")
#     i -= 1

# # Creazione del DataFrame
# df = pd.DataFrame(results)
# df.shape

In [10]:
# df.drop_duplicates(inplace=True)
# df['taken_at'] = pd.to_datetime(df['taken_at'], unit='s')
# df.to_csv("data.csv", index=False)

In [11]:
df = pd.read_csv("followers_Artificial_Intelligence.csv")
df['follower_pk'] = df['follower_pk'].astype(str)
unique_user_pk = df['follower_pk'].unique()
len(unique_user_pk)

15571

In [12]:
def load_processed_users(file_path):
    if os.path.exists(file_path):
        logging.info(f"Loading processed users from {file_path}")
        df = pd.read_csv(file_path)
        df['user_pk'] = df['user_pk'].astype(str)
        return set(df['user_pk'].tolist())
    return set()

def save_to_csv(df, file_path):
    try:
        if os.path.exists(file_path):
            df.to_csv(file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(file_path, index=False)
        logging.info(f"Saved {len(df)} records to {file_path}")
    except Exception as e:
        logging.error(f"Error saving data to {file_path}: {e}")
        
csv_file = "followers_followers_Artificial_Intelligence.csv"
processed_users = load_processed_users(csv_file)

In [13]:
len(processed_users)

539

In [14]:
print(len(unique_user_pk))
unique_user_pk = unique_user_pk[unique_user_pk != "45367404464"]
print(len(unique_user_pk))

15571
15570


In [15]:
def fetch_followers(pk):
    print(f"Fetching followers for user_pk: {pk}")
    try:
        follower_data = get_all_follower(pk)
        return [
            {
                "user_pk": pk,
                "follower_pk": follower["pk"],
                "follower_username": follower["username"],
                "follower_count": follower["follower_count"]
            }
            for follower in follower_data
        ]
    except Exception as e:
        logging.error(f"Error fetching followers for {pk}: {e}")
        return None



follower_list = []
batch_size = 1


print("Starting follower data collection")
for user_pk in unique_user_pk:
    if user_pk in processed_users:
        print(f"Skipping already processed user_pk: {user_pk}")
        continue
    
    processed_users.update([user_pk])
    results = fetch_followers(user_pk)
    if results is None:
        break
    follower_list.extend(results)
    
    if follower_list:
        print(f"Processing batch of {len(follower_list)} followers")
        follower_df = pd.DataFrame(follower_list)
        save_to_csv(follower_df, csv_file)
        follower_list = []

if follower_list:
    print(f"Saving remaining {len(follower_list)} followers")
    follower_df = pd.DataFrame(follower_list)
    save_to_csv(follower_df, csv_file)

print("Follower data collection completed")

Starting follower data collection
Skipping already processed user_pk: 1451797369
Skipping already processed user_pk: 2384555054
Skipping already processed user_pk: 17096723
Skipping already processed user_pk: 55856570272
Skipping already processed user_pk: 33147008532
Skipping already processed user_pk: 3702020941
Skipping already processed user_pk: 472676306
Skipping already processed user_pk: 32185561361
Skipping already processed user_pk: 13534725
Skipping already processed user_pk: 1727625343
Skipping already processed user_pk: 8006990
Skipping already processed user_pk: 1735244853
Skipping already processed user_pk: 8960413271
Skipping already processed user_pk: 39436106246
Skipping already processed user_pk: 2343042146
Skipping already processed user_pk: 3584651501
Skipping already processed user_pk: 1594819278
Skipping already processed user_pk: 2098126112
Skipping already processed user_pk: 33063492581
Skipping already processed user_pk: 2034417720
Skipping already processed us

ERROR:root:Error fetching followers for 1796326818: Errore generale nel recupero dei follower per l'utente 1796326818: Errore nel recupero della paginazione dei follower: 'NoneType' object has no attribute 'get'


Follower data collection completed


In [16]:
df1 = pd.read_csv(csv_file)
df_count= df1.drop_duplicates().groupby("user_pk").count()
print(df_count.shape)
df_count

(539, 3)


,follower_pk,follower_username,follower_count
user_pk,,,
269009,50,50,50
301440,50,50,50
864043,25,25,25
1885959,50,50,50
3347806,50,50,50
...,...,...,...
70296375138,50,50,50
70800366460,1,0,0
71134731317,3,3,3


In [18]:
import requests

url = "https://www.threads.net/graphql/query"

# Header senza i pseudo-header HTTP/2
headers = {
    "accept": "*/*",
    "accept-language": "it-IT,it;q=0.7",
    "content-type": "application/x-www-form-urlencoded",
    "cookie": 'cb=1_1970_01_01_2-3; mid=Z6O-5AALAAFwri_hZd_yj-u2-2EC; ig_did=6A6269B0-9F68-4E7C-9CEA-53AD0784C3B1; dpr=1; csrftoken=UgpugvagTtbii6zdkqFUduvZ1EVh1ezX; ds_user_id=65955050144; sessionid=65955050144%3A6hS26w11VRkHbU%3A24%3AAYeTigtAMpGXCp8sOwNzqCe29YQYygcEAfegg7eTjg',
    "origin": "https://www.threads.net",
    "priority": "u=1, i",
    "sec-ch-ua": '"Not A(Brand";v="8", "Chromium";v="132", "Brave";v="132"',
    "sec-ch-ua-full-version-list": '"Not A(Brand";v="8.0.0.0", "Chromium";v="132.0.0.0", "Brave";v="132.0.0.0"',
    "sec-ch-ua-mobile": "?1",
    "sec-ch-ua-model": "\"Nexus 5\"",
    "sec-ch-ua-platform": "\"Android\"",
    "sec-ch-ua-platform-version": "\"6.0\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "sec-gpc": "1",
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Mobile Safari/537.36",
    "x-asbd-id": "129477",
    "x-bloks-version-id": "c16e9c7c1a812f60d22c42530908301bfdaebb593260b5e4c267e3b8371a1113",
    "x-csrftoken": "4VCiPzU8CuE1aB1zL9x6oPYfvZwF8116",
    "x-fb-friendly-name": "BarcelonaFriendshipsFollowersTabRefetchableQuery",
    "x-fb-lsd": "GJRvxvEEcqbVrIhzC1FcF4",
    "x-ig-app-id": "1412234116260832"
}

# Corpo della richiesta (i parametri form-urlencoded)
data = {
    "av": "17841471788496113",
    "__user": "0",
    "__a": "1",
    "__req": "3n",
    "__hs": "20124.HYP:barcelona_web_pkg.2.1...1",
    "dpr": "2",
    "__ccg": "UNKNOWN",
    "__rev": "1019856045",
    "__s": "6oyndv:4euxjq:sxijxn",
    "__hsi": "7468022932953406426",
    "__dyn": "7xeUmwlEnwn8K2Wmh0no6u5U4e0yoW3q32360CEbo1nEhw2nVE4W0qa0FE2awgo9oO0n24oaEd82lwv89k2C1Fwc60D85m1mzXwae4UaEW0Loco5G0zK5o4q0HU1IEGdwtU2ewbS1LwTwKG0hq1Iwqo9EpwUwiQ1mwLwHxW5Ec8yu4Q2-qfwGwaKbw",
    "__csr": "grF98ybbYmyl_lGDf8j5dYjCHTAGWyqFa8WgSi5e8G5EhDAyFECey8Gm-u26mi9w04_zw9fw3783Hw5cx65Q6p7p1Mkwmax6t2Ga223x0191341ix20ciUlw3jXjh4ewlo1ao7Cq0DE8cZ01h66yJH8Vx05c4UjOyQy4DyVGzQ1fHc0CXFo6C4kahMC2Ikt2H80a2UJadgfs8wGwk0E951gldg2vwyx8w63g",
    "__comet_req": "29",
    "fb_dtsg": "NAcNI51-L_PxXxGvOJ9z8b0RJbKo14I0t99DDJmcb0dO7Nc3C5AS5qg:17865068956001195:1738784583",
    "jazoest": "25840",
    "lsd": "GJRvxvEEcqbVrIhzC1FcF4",
    "__spin_r": "1019856045",
    "__spin_b": "trunk",
    "__spin_t": "1738784586",
    "__jssesw": "2",
    "fb_api_caller_class": "RelayModern",
    "fb_api_req_friendly_name": "BarcelonaFriendshipsFollowersTabRefetchableQuery",
    "variables": '{"after":"30","first":10,"id":"71800877099","__relay_internal__pv__BarcelonaIsLoggedInrelayprovider":true,"__relay_internal__pv__BarcelonaIsCrawlerrelayprovider":false,"__relay_internal__pv__BarcelonaHasDisplayNamesrelayprovider":false}',
    "server_timestamps": "true",
    "doc_id": "9226067564176291"
}

# Eseguiamo la richiesta POST
response = requests.post(url, headers=headers, data=data)

# Visualizziamo il codice di stato e il contenuto della risposta
print("Status code:", response.status_code)
print("Response body:")
print(response.text)


Status code: 200
Response body:
{"data":{"fetch__XDTUserDict":{"followers":{"edges":[{"node":{"pk":"9215079294","text_post_app_is_private":true,"friendship_status":{"followed_by":false,"following":false,"outgoing_request":false},"profile_pic_url":"https://scontent-fco2-1.cdninstagram.com/v/t51.2885-19/472599241_1126809945174980_6470614131264216941_n.jpg?stp=dst-jpg_s150x150_tt6\u0026_nc_ht=scontent-fco2-1.cdninstagram.com\u0026_nc_cat=108\u0026_nc_ohc=0A2gJupV4McQ7kNvgHATCDx\u0026_nc_gid=ed9c63d3b7cd45a8acef1060ee7dc713\u0026edm=AIYyWWoBAAAA\u0026ccb=7-5\u0026oh=00_AYCYwii-vJd1Q6m0UFE6h2acChgeg54-S9KDj1d3dLDBJw\u0026oe=67A9983D\u0026_nc_sid=57eee7","username":"luigi.mondo","follower_count":13,"profile_context_facepile_users":[],"is_verified":false,"full_name":"Luigi Mondo","has_onboarded_to_text_post_app":true,"id":"9215079294","__typename":"XDTUserDict"},"cursor":null},{"node":{"pk":"7854446164","text_post_app_is_private":false,"friendship_status":{"followed_by":false,"following":fals